In [1]:
import os
from pathlib import Path
import sys

# set working directory to the module's home
cwd = os.getcwd()
os.chdir(Path(cwd).parents[1])

In [75]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from config import connect
from db_management.database_management import load_data, create_table, csv_to_rows
from modeling.time_series_forecast import TimeSeriesForecaster

### Connect to MySQL Database `AQI` for the Table `combined`

In [3]:
conn = connect()
cursor = conn.cursor()

In [46]:
df = load_data(conn, "AQI", "index", "SELECT state_name, date, aqi FROM AQI.index")
df.head()

,state_name,date,aqi
0,Alabama,2021-01-01,30
1,Alabama,2021-01-02,27
2,Alabama,2021-01-03,34
3,Alabama,2021-01-04,36
4,Alabama,2021-01-05,31


In [47]:
df["date"] = pd.to_datetime(df["date"])

In [48]:
df.head()

,state_name,date,aqi
0,Alabama,2021-01-01,30
1,Alabama,2021-01-02,27
2,Alabama,2021-01-03,34
3,Alabama,2021-01-04,36
4,Alabama,2021-01-05,31


In [78]:
forecast_df = pd.DataFrame()

for state in df.state_name.unique():
    state_df = df[df.state_name==state]
    tsf = TimeSeriesForecaster(mcmc_samples=100)
    tsf.fit(df, "date", "aqi", "MS", "mean")
    fcst = tsf.forecast(periods=120)
    fcst["state"] = state
    forecast_df = forecast_df.append(fcst)

In [59]:
forecast_df

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat,y,state
0,1980-01-01,57.816903,31.478991,55.229076,55.637355,60.024801,-0.244466,-0.302716,-0.186940,-0.244466,-0.302716,-0.186940,0.0,0.0,0.0,43.682623,42.0,Alabama
1,1980-02-01,57.741065,34.564228,59.144237,55.591264,59.934123,-0.188964,-0.235461,-0.139482,-0.188964,-0.235461,-0.139482,0.0,0.0,0.0,46.830109,48.0,Alabama
2,1980-03-01,57.670121,38.622778,63.203764,55.547059,59.853500,-0.114709,-0.162750,-0.066549,-0.114709,-0.162750,-0.066549,0.0,0.0,0.0,51.054858,46.0,Alabama
3,1980-04-01,57.594283,47.555183,72.003652,55.499806,59.770320,0.052176,-0.008002,0.115135,0.052176,-0.008002,0.115135,0.0,0.0,0.0,60.599297,51.0,Alabama
4,1980-05-01,57.520892,58.443849,82.709081,55.449537,59.688180,0.235300,0.183027,0.291279,0.235300,0.183027,0.291279,0.0,0.0,0.0,71.055551,58.0,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,2030-08-01,29.413576,26.271862,41.295748,25.775072,33.012658,0.143278,0.106333,0.178814,0.143278,0.106333,0.178814,0.0,0.0,0.0,33.627895,NaN,South Dakota
545,2030-09-01,29.380918,21.561033,36.587389,25.690974,32.998174,-0.009246,-0.043118,0.023051,-0.009246,-0.043118,0.023051,0.0,0.0,0.0,29.109261,NaN,South Dakota
546,2030-10-01,29.349313,19.318808,33.546939,25.642003,32.983233,-0.100989,-0.138602,-0.063989,-0.100989,-0.138602,-0.063989,0.0,0.0,0.0,26.385354,NaN,South Dakota
547,2030-11-01,29.316655,18.701306,33.003240,25.555368,32.989909,-0.111536,-0.142763,-0.077273,-0.111536,-0.142763,-0.077273,0.0,0.0,0.0,26.046783,NaN,South Dakota
